In [ ]:
# импортируем нужные библиотеки

from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import torch
import json

In [ ]:
class CustomDataset(Dataset):
    """  класс для хранения датасета и его предоработки для формата ruBert"""
    def __init__(self, texts, targets, tokenizer, max_len=512):
        """
        text: list, список текстов для обучения модели;
        target: list, список целочисленных предсказаний для обучения модели;
        tokenizer: токенизатор для предобработки текста;
        max_len: максимальное количество слов в тексте.
        """
        self.texts = texts
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        """ метод, возвращающий размер датасета. Нужен для создания DataLoader'а """
        return len(self.texts)

    def __getitem__(self, idx):
        """ Метод для получения объекта датасета под номером idx
            Принимает на вход: idx, int индекс желаемого элемента
            Взовращает: dict с ключами:
                -text: str текст;
                -inputs_ids: закодированный с помощью токенизатора в список идентефикаторов текст;
                -attention_mask: закодированный с помощью токенизатора в список масок текст;
                -targets: номер класса, которому принадлежит объект.
            """
        text = str(self.texts[idx])
        target = self.targets[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
          'text': text,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'targets': torch.tensor(target, dtype=torch.long)
        }

In [ ]:
class BertClassifier:
    """ класс для инициализации, обучения и получения предсказаний ruBert модели """
    def __init__(self, model_path, tokenizer_path, n_classes=4, epochs=10, model_save_path='berts_training_weights'):
        """
        model_path: путь до предобученной модели;
        tokenizer_path: путь до используемого токенизатора;
        n_classes: количество классов классификации;
        epochs: количество эпох обучения;
        model_save_path: путь, куда будут сохранятся чекпоинты весов во время обучения.
        """
        self.model = BertForSequenceClassification.from_pretrained(model_path)
        self.tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model_save_path=model_save_path
        self.max_len = 512
        self.epochs = epochs
        self.out_features = self.model.bert.encoder.layer[1].output.dense.out_features
        self.model.classifier = torch.nn.Linear(self.out_features, n_classes)
        self.model.to(self.device)
    
    def preparation(self, X_train, y_train, X_valid, y_valid):
        """ Метод для получения данных для обучения и валидации, 
            превращения их в pytorch DataLoader'ы и инициализации функции потерь и изменения learning rat'а.
            Принимает на вход: 
                -X_train, y_train: list, тренировочная выборка и предсказания к ней;
                -X_valid, y_valid: list, валидационная выборка и предсказания к ней."""
        self.train_set = CustomDataset(X_train, y_train, self.tokenizer)
        self.valid_set = CustomDataset(X_valid, y_valid, self.tokenizer)

        # сделаем из датасетов pytorch DataLoader'ы
        self.train_loader = DataLoader(self.train_set, batch_size=2, shuffle=True)
        self.valid_loader = DataLoader(self.valid_set, batch_size=2, shuffle=True)

        # инициализуем оптимизаторы для обучения
        self.optimizer = AdamW(self.model.parameters(), lr=2e-5, correct_bias=False)
        self.scheduler = get_linear_schedule_with_warmup(
                self.optimizer,
                num_warmup_steps=0,
                num_training_steps=len(self.train_loader) * self.epochs
            )
        self.loss_fn = torch.nn.CrossEntropyLoss().to(self.device)
            
    def fit(self):
        """ метод для обучения модели на тренировочной выборке"""
        self.model = self.model.train()
        losses = []
        correct_predictions = 0

        for data in self.train_loader:
            input_ids = data["input_ids"].to(self.device)
            attention_mask = data["attention_mask"].to(self.device)
            targets = data["targets"].to(self.device)

            outputs = self.model(
                input_ids=input_ids,
                attention_mask=attention_mask
                )

            preds = torch.argmax(outputs.logits, dim=1)
            loss = self.loss_fn(outputs.logits, targets)

            correct_predictions += torch.sum(preds == targets)

            losses.append(loss.item())
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            self.optimizer.step()
            self.scheduler.step()
            self.optimizer.zero_grad()

        train_acc = correct_predictions.double() / len(self.train_set)
        train_loss = np.mean(losses)
        return train_acc, train_loss
    
    def eval(self):
        """ метод для проверки точности модели на валидационной выборки """
        self.model = self.model.eval()
        losses = []
        correct_predictions = 0

        with torch.no_grad():
            for data in self.valid_loader:
                input_ids = data["input_ids"].to(self.device)
                attention_mask = data["attention_mask"].to(self.device)
                targets = data["targets"].to(self.device)

                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask
                    )

                preds = torch.argmax(outputs.logits, dim=1)
                loss = self.loss_fn(outputs.logits, targets)
                correct_predictions += torch.sum(preds == targets)
                losses.append(loss.item())
        
        val_acc = correct_predictions.double() / len(self.valid_set)
        val_loss = np.mean(losses)
        return val_acc, val_loss
    
    def train(self):
        """ метод для обучения модели """
        best_accuracy = 0
        for epoch in range(self.epochs):
            print(f'Epoch {epoch + 1}/{self.epochs}')
            train_acc, train_loss = self.fit()
            print(f'Train loss {train_loss}| accuracy {train_acc}')

            val_acc, val_loss = self.eval()
            print(f'Val loss {val_loss}| accuracy {val_acc}')
            print('-' * 10)

            if val_acc > best_accuracy:
                torch.save(self.model, self.model_save_path + "/" + "epoch" + str(epoch))
                best_accuracy = val_acc

        self.model = torch.load(self.model_save_path + "/final_model.pt")
    
    def predict(self, text: str) -> int:
        """ 
        метод для получения предсказания модели на тексте
        Принимает на вход: str, текст;
        Возвращает: int, номер предсказанного класса.
        """
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )
        
        out = {
              'text': text,
              'input_ids': encoding['input_ids'].flatten(),
              'attention_mask': encoding['attention_mask'].flatten()
          }
        
        input_ids = out["input_ids"].to(self.device)
        attention_mask = out["attention_mask"].to(self.device)
        
        outputs = self.model(
            input_ids=input_ids.unsqueeze(0),
            attention_mask=attention_mask.unsqueeze(0)
        )
        
        prediction = torch.argmax(outputs.logits, dim=1).cpu().numpy()[0]

        return prediction

In [ ]:
# инициализируем модель ruBert с предобученными весами

classifier = BertClassifier(
        model_path='cointegrated/rubert-tiny',
        tokenizer_path='cointegrated/rubert-tiny',
        n_classes=4,
        epochs=30
)

In [ ]:
dataset = pd.read_csv("submission_with_rubert.csv")

In [ ]:
dataset.head()

In [ ]:
# загрузим собранный датасет

dataset = json.load(open("big_dataset.json", "r"))

texts = list(dataset.keys())
labels = list(dataset.values())
labelEncoder = {"спорт": 0, "музыка": 1, "литература": 2, "животные": 3}

texts = list(map(lambda x: x[:min(513, len(x))], texts)) # обрежем текст до первых 512 слов
labels = list(map(lambda x: labelEncoder[x], labels)) # преобразуем строки со значениями классов в целочисленный формат

In [ ]:
# разобьём выборку на обучающую и валидационную

X_train, X_valid, y_train, y_valid = train_test_split(texts, labels, 
                                                      test_size=0.1, random_state=42)

In [ ]:
# загрузим датасет в классификатор

classifier.preparation(
        X_train,
        y_train,
        X_valid,
        y_valid)

In [ ]:
# дообучим классификатор на загруженном датасете

classifier.train()

In [ ]:
# попробуем осуществить предсказание с помощью дообученной модели

In [ ]:
labelDecoder = {v: k for k, v in labelEncoder.items()}

In [ ]:
labelDecoder[classifier.predict("""Реши задачу. Сколько коробок 
                   корма для собак поместится в ящике, 
                   если все стороны ящика соответственно в 12 раз больше сторон коробки корма?""")]

In [ ]:
torch.save(classifier.model, "theme_classifier.pt")

### Выполним предсказания для тестовой выборки

In [ ]:
# импортируем функцию для нахождения самых близкоотносящихся к теме слов

from get_most_similarity_words import getMostSimilarityWords

In [ ]:
test_dataset = pd.read_csv("test.csv")
sample_submission = pd.read_csv("sample_submission.csv")

In [ ]:
test_predicts = []
keywords = []

for text in test_dataset["task"]:
    p = labelDecoder[classifier.predict(text)]
    test_predicts.append(p)
    keywords.append(";".join(getMostSimilarityWords(text, p)))

In [ ]:
sample_submission['category'] = test_predicts
sample_submission["keywords"] = keywords

In [ ]:
sample_submission.head()

In [ ]:
sample_submission.to_csv("submission_with_rubert.csv")